In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# Optimizers

In [2]:
#export
from exp.nb_09a import *

## Data

In [4]:
path = datasets.untar_data(datasets.URLs.IMAGENETTE_160) # downloads and returns a path to folder
tfms = [make_rgb, ResizeFixed(128), to_byte_tensor, to_float_tensor] # transforms to be applied to images

bs = 128

il = ImageList.from_files(path, tfms=tfms) # Imagelist from files
sd = SplitData.split_by_func(il, partial(grandparent_splitter, valid_name="val")) # Splitdata by function
ll = label_by_func(sd, parent_labeler, proc_y=CategoryProcesser()) # label the data by parent folder
data = ll.to_databunch(bs, c_in=3, c_out=10)

In [5]:
nfs = [32,64,128,256]

In [6]:
callbacks = [partial(AvgStatsCallback, accuracy),
            CudaCallback, 
            partial(BatchTransformXCallback, norm_imagenette)]

In [11]:
learn, run = get_learn_run(data, nfs,conv_layer, 0.4,  cbs=callbacks)

Baseline single epoch validation accuracy with vanilla Stochastic Gradient Descent:

In [12]:
run.fit(1, learn)

train: [1.7702564783768084, tensor(0.3910, device='cuda:0')]
valid: [1.6495183121019108, tensor(0.4525, device='cuda:0')]


## Refining the Optimizer

## Weight Decay

## Momentum

## Adam

## Lamb

In [ ]:
nb_auto_export()